In [2]:
# Description: Load tsv file

# Import libraries
import pandas as pd


# Load tsv file
News = pd.read_csv('MINDsmall_train/news.tsv', sep='\t', header=None)
News.columns = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']

User = pd.read_csv('MINDsmall_train/behaviors.tsv', sep='\t', header=None)
User.columns = ['impression_id', 'user_id', 'time', 'history', 'impressions']

topic_size = News['category'].nunique()
subtopic_size = News['subcategory'].nunique()

print(f"Data contains {topic_size} topics and {subtopic_size} subtopics")

Data contains 17 topics and 264 subtopics


In [3]:
from LSTUR import NewsEncoder, TitleEncoder, TopicEncoder, GloVe
import torch as th
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

topic_embedding_dim = 100
subtopic_embedding_dim = 100
word_size = 100


NewsEncoderModule = NewsEncoder(100, 100, topic_size, subtopic_size, 10000)
TitleEncoderModule = TitleEncoder(100)
TopicEncoderModule = TopicEncoder(100, 100, topic_size, subtopic_size)

topic = th.tensor([0],dtype=th.int32)
subtopic = th.tensor([0],dtype=th.int32)

title = tokenizer(News['title'][0])
W = GloVe.get_vecs_by_tokens(title)

C:\Users\Nikolaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Encode categories and subcategories:
categories = News['category'].unique()
subcategories = News['subcategory'].unique()

test = [i for i in range(len(categories))]
test2 = [i for i in range(len(subcategories))]

# Add encodings to dict
res = {}
for cat in categories:
    for value in test:
        res[cat] = value
        test.remove(value)
        break


res2 = {}
for subcat in subcategories:
    for value in test2:
        res2[subcat] = value
        test2.remove(value)
        break


# Replace column with encodings:
C = th.tensor(News['category'].replace(res))
SC = th.tensor(News['subcategory'].replace(res2))

print(C.shape)
print(SC.shape)




torch.Size([51282])
torch.Size([51282])


In [99]:
# Get titles lengths
import numpy as np
from torch.nn.utils.rnn import pad_sequence

lengths = th.tensor([len(tokenizer(News['title'][i])) for i in range(News['title'].size)])


# Pad titles
maxlength = max(lengths)
padlengths = (maxlength-lengths).tolist()
padtitles = [tokenizer(News["title"][i] + " <Pad>"*padlengths[i]) for i in range(len(padlengths))]



# Embed titles
Article_embedding = th.zeros(len(padtitles),maxlength,300)
i = 0
for t in padtitles:
    Article_embedding[i] = GloVe.get_vecs_by_tokens(t)
    i+=1




In [114]:
# Dictionary of articles from their ID's
newsid = News['news_id']
article_dict = {newsid[i]:Article_embedding[i] for i in range(len(padtitles))}

print(len(article_dict))
print(article_dict["N55189"].shape)


51282
torch.Size([64, 300])


In [127]:
# User ID's of each impression log:
users = User['user_id'].unique()

test2 = [i for i in range(len(users))]


# Add encodings to dict
userid_dict = {}
for user in users:
    for value in test2:
        userid_dict[user] = value
        test2.remove(value)
        break



# Replace column with encodings:
U = th.tensor(User['user_id'].replace(userid_dict))
U.shape

KeyboardInterrupt: 

In [243]:
# For each impression log: all topics, subtopics and titles embedded and encoded in tensors
# First pad topic/subtopic sequences
N = len(User['history'])


# th.zeros()
seq_lengths = th.zeros(N)
for i in range(N):
    if isinstance(User['history'][i], str):
        seq_lengths[i] = len(tokenizer(User['history'][i]))
seq_lengths

# [item.upper() for item in test]

# seq_lengths = [(len(tokenizer(User['history'][i]))) for i in range(len(User['history']))]

# Find max sequence length
max_seq_length = max(seq_lengths)


pad_seq_lengths = ((max_seq_length-seq_lengths).to(th.int32)).tolist()

X = [item.upper() for item in User["history"][0]]
# print(tokenizer(User["history"][0]) + ["Hpad>>"]*2)



# NOT WORKING YET: NEED to encode history first
pad_hists = th.zeors(N,max_seq_length)
for i in range(N):
pad_hists = [th.tensor(tokenizer(User["history"][0]) + ["<HPad>"]*pad_seq_lengths[i]) ]

pad_hists



['n55189', 'n42782', 'n34694', 'n45794', 'n18445', 'n63302', 'n10414', 'n19347', 'n31801', 'Hpad>>', 'Hpad>>']


ValueError: too many dimensions 'str'

In [ ]:
NewsEncoderModule(topic, subtopic, W).size()

In [229]:
tokenizer(User["history"][0])

['n55189',
 'n42782',
 'n34694',
 'n45794',
 'n18445',
 'n63302',
 'n10414',
 'n19347',
 'n31801']

In [226]:
(max_seq_length-seq_lengths).to(th.int32)

tensor([549, 476, 542,  ..., 535, 500, 556], dtype=torch.int32)